# Docking Oracle and Generative Leaderboard Demo

Wenhao Gao (whgao@mit.edu)

# Outline
- Docking oracle
    - what is docking
    - Why we prefer docking
    - how to use docking oracle in TDC
- generative leaderboard
    - docking benchmark group
    - Train your own model with docking oracle
    - Submit to leaderboard

## Molecular design is an optimization problem

<img src="oracle.png" width="600"/>

## Docking Oracle

* What is docking?

Molecular docking is a computational approach to evaluate the binding affinity of a ligand (small molecule, flexible) and a receptor (protein target, rigid). Typically it includes two components: a scoring function to evaluate the free energy of a determined conformation, and a searching algorithm that search for the stronger binding conformation. A stronger binding affinity leads to higher probability to be a potent inhibiter (recall the lock and key model).

Bender, B. J., Gahbauer, S., Luttens, A., Lyu, J., Webb, C. M., Stein, R. M., ... & Shoichet, B. K. (2021). A practical guide to large-scale docking. Nature Protocols, 1-34.

<img src="docking.png" width="1000"/>

* Why we use docking?

1. Docking is actually used as a first step in virtual screening for drug discovery, more useful and challenging.
2. Docking serves as an affordable simulation of resource-consuming oracles (docking: ~30s/mol, QED: ms/mol).

<img src="why_docking.png" width="800"/>

## How to run docking (traditional vs. TDC) ?

Traditional docking simulation: https://vina.scripps.edu/manual/#linux

<img src="vina.png" width="800"/>

<img src="docking_gflownet.png" width="800"/>

We provide easy access with AutoDock Vina as backend via PyScreener, install PyScreener following https://github.com/coleygroup/pyscreener

We provide 9 curated targets: PDB ID: 3pbl, 1iep, 2rgp, 3eml, 3ny8, 4rlu, 4unn, 5mo4, 7l11. Using them is similar to using other oracles:

| PDB ID      | Description | Source      |
| ----------- | ----------- | ----------- |
| [1iep](https://www.rcsb.org/structure/1IEP)        | c-Abl kinase, complex with imatinib. | [vina tutorial](https://autodock-vina.readthedocs.io/en/latest/docking_python.html) |
| [2rgp](https://www.rcsb.org/structure/2RGP)        | Epidermal growth factor receptor, complex with hydrazone.        | [SBMolGen](https://chemrxiv.org/engage/chemrxiv/article-details/60c75725842e65c7acdb4638)        |
| [3eml](https://www.rcsb.org/structure/3EML)        | Human A2A adenosine receptor, complex with ZM241385.        | [SBMolGen](https://chemrxiv.org/engage/chemrxiv/article-details/60c75725842e65c7acdb4638)        |
| [3ny8](https://www.rcsb.org/structure/3NY8)        | Beta2 adrenergic receptor, complex with the inverse agonist ICI 118,551.        | [SBMolGen](https://chemrxiv.org/engage/chemrxiv/article-details/60c75725842e65c7acdb4638)        |
| [3pbl](https://www.rcsb.org/structure/3PBL)        | Human dopamine D3 receptor, complex with eticlopride.        | [DUD.E](http://dude.docking.org/targets/drd3)        |
| [4rlu](https://www.rcsb.org/structure/4RLU)        | beta-hydroxyacyl-ACP dehydratase HadAB, complex with a flavonoid inhibitor.       | [A 3D generative model for structure-based drug design](https://proceedings.neurips.cc/paper/2021/hash/314450613369e0ee72d0da7f6fee773c-Abstract.html)        |
| [4unn](https://www.rcsb.org/structure/4UNN)        | M. Tuberculosis Thymidylate Kinase (Mtb Tmk), complex witha. inhibitor.        | [MolPAL](https://pubs.rsc.org/en/content/articlehtml/2021/sc/d0sc06805e)        |
| [5mo4](https://www.rcsb.org/structure/5MO4)        | ABL1 kinase, complex with asciminib.        | [L-Net](https://arxiv.org/abs/2104.08474)        |
| [7l11](https://www.rcsb.org/structure/7L11)        | The Main Protease (M$^{pro}$) of SARS-CoV-2, complex with a known inhibitor.        | [Zhang, Chun-Hui, et al.](https://pubs.acs.org/doi/abs/10.1021/acscentsci.1c00039)        |

In [1]:
from tdc import Oracle
import time

oracle = Oracle(name='3pbl_docking')
t1 = time.time()
print(f"Docking score: {oracle('CCNC(=O)c1ccc(NC(=O)N2CC[C@H](C)[C@H](O)C2)c(C)c1')} kcal/mol")
print(f"Consumed time: {time.time() - t1:.3f} s")

Found local copy...
2022-01-24 23:17:41,348	INFO services.py:1253 -- View the Ray dashboard at http://127.0.0.1:8266
Docking: 100%|██████████| 1/1 [00:23<00:00, 23.08s/ligand]


Docking score: -8.0 kcal/mol
Consumed time: 23.086 s


## Visualize the result

In [2]:
import py3Dmol
import os

pdbid = '3pbl'

# conda install openbabel -c conda-forge
os.system("obabel -i pdbqt out.pdbqt -o pdb -O " + pdbid + "_ligand.pdb")
os.system("obabel -i pdbqt oracle/" + pdbid + ".pdbqt -o pdb -O" + pdbid + "_receptor.pdb")
os.system("obabel -i pdbqt out.pdbqt -o xyz -O " + pdbid + "_ligand.xyz")

0

In [3]:
lines = []
for line in open(pdbid + "_ligand.xyz", 'rt'): 
    lines.append(line.strip())
lines = lines[2:]

xs = [float(line.split()[1]) for line in lines]
ys = [float(line.split()[2]) for line in lines]
zs = [float(line.split()[3]) for line in lines]
center_x = sum(xs)/len(lines)
center_y = sum(ys)/len(lines)
center_z = sum(zs)/len(lines)
print(center_x, center_y, center_z)

def visbox2(objeto, center, box_size):  
    objeto.addBox({'center':{'x': center[0],'y': center[1],'z': center[2]}, 
                   'dimensions': {'w': box_size[0],'h': box_size[1],'d': box_size[2]},'color':'blue','opacity': 0.5})

def complxvis(objeto,protein_name,ligand_name):
    mol1 = open(protein_name, 'r').read()
    mol2 = open(ligand_name, 'r').read()
    objeto.addModel(mol1,'pdb')
    objeto.setStyle({'cartoon': {'color':'spectrum'}})
    objeto.addModel(mol2,'pdb')
    objeto.setStyle({'model':1},{'stick':{}})

def vismol(center=[center_x, center_y, center_z], box_size=[20, 20, 20]):  
    mol_view = py3Dmol.view(1200, 800)  
    visbox2(mol_view, center, box_size)
    complxvis(mol_view, pdbid + '_receptor.pdb', pdbid + '_ligand.pdb')
    mol_view.setBackgroundColor('white')
    mol_view.rotate(90, {'x':0,'y':1,'z':0},viewer=(0,1));
    mol_view.zoomTo()  
    mol_view.show()

vismol()

7.394807692307693 22.16196153846154 23.601230769230767


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Others

In [4]:
oracle = Oracle(name='7l11_docking')
t1 = time.time()
print(f"Docking score: {oracle('CCNC(=O)c1ccc(NC(=O)N2CC[C@H](C)[C@H](O)C2)c(C)c1')} kcal/mol")
print(f"Consumed time: {time.time() - t1:.3f} s")

Found local copy...
2022-01-24 23:18:09,718	INFO services.py:1253 -- View the Ray dashboard at http://127.0.0.1:8266
Docking:   0%|          | 0/1 [00:00<?, ?ligand/s]

(prepare_receptors pid=2319330) adding gasteiger charges to peptide


Docking: 100%|██████████| 1/1 [00:24<00:00, 24.72s/ligand]


Docking score: -5.9 kcal/mol
Consumed time: 24.728 s


In [5]:
t1 = time.time()
print(f"Docking score: {oracle(['CCNC(=O)c1ccc(NC(=O)N2CC[C@H](C)[C@H](O)C2)c(C)c1', 'CCNC(=O)c1ccc(NC(=O)N2CC[C@H](C)[C@H](O)C2)c(C)c1'])} kcal/mol")
print(f"Consumed time: {time.time() - t1:.3f} s")

Docking: 100%|██████████| 1/1 [00:24<00:00, 24.24s/ligand]


Docking score: [-6.4, -6.0] kcal/mol
Consumed time: 48.272 s


We provide a simple interface to build such an oracle with custom protein target

In [6]:
# oracle = Oracle(name='pyscreener', receptor_pdb_file=, box_center=, box_size=, software_class='vina', ncpu=4)
# oracle('c1ccccc1')

We also have a backend of AutoDock Vina, please install vina following https://autodock-vina.readthedocs.io/en/latest/installation.html#python-bindings

In [7]:
# oracle_vina = Oracle(name='3pbl_docking_vina')
# t1 = time.time()
# print(f"Docking score: {oracle_vina('CCNC(=O)c1ccc(NC(=O)N2CC[C@H](C)[C@H](O)C2)c(C)c1', output_file='out.pdbqt', n_poses=1)} kcal/mol")
# print(f"Consumed time: {time.time() - t1:.3f} s")

## Generative Modeling Leaderboard

[TDC generative leaderboard](https://tdcommons.ai/benchmark/docking_group/drd3/)

<img src="leaderboard_generative.png" width="800"/>

### Step 1: Initialize the docking benchmark group class

In [8]:
from tdc import utils
utils.retrieve_benchmark_names('Docking_Group')

['1iep', '2rgp', '3eml', '3ny8', '4rlu', '4unn', '5mo4', '7l11', '3pbl']

In [9]:
from tdc.benchmark_group import docking_group
group = docking_group(path='./data')

benchmark = group.get('3pbl', num_max_call = 5000) 
oracle_fct, data, name = benchmark['oracle'], benchmark['data'], benchmark['name'] 

Found local copy...
Found local copy...
2022-01-24 23:19:27,496	INFO services.py:1253 -- View the Ray dashboard at http://127.0.0.1:8266


In [10]:
data.head()

,smiles
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...


In [11]:
oracle_fct(data.smiles[0])

Docking: 100%|██████████| 1/1 [00:24<00:00, 24.91s/ligand]


-8.4

In [12]:
name

'3pbl'

### Step 2: Train your model (a genetic algorithm with SynNet)

Gao, W., Mercado, R., & Coley, C. W. Amortized Tree Generation for Bottom-up Synthesis Planning and Synthesizable Molecular Design. ICLR 2022.

<img src="generation_process.png" width="800"/>

<img src="ga_illustration.pdf" width="600"/>

In [13]:
import json

with open('opt_drd3_5000.json', 'r') as f:
    pred_smiles = json.load(f)

In [14]:
pred_smiles

{'5000': {'O=c1c2c(F)c(-c3cccc4ccccc34)ccc2ncn1CC12CCC(CC1)OC2': 12.3,
  'O=c1c2c(Br)cccc2ncn1Cc1cc(F)c(-c2n~c3c(C4=NNN=N4)cccc3o2)cc1F': 12.2,
  'O=C1OC(=O)C23CCOCC12N=NN3CC12CC(C3=NC(c4cncc5ccccc45)=NN3)(CO1)C2': 12.2,
  'FC(F)(F)c1cncc(CN2CCc3cc(-c4cccc5ccccc45)ccc32)c1': 12.0,
  'Cc1cc(N)n2ncc(-c3nc(-c4ccc(-c5cccc6ccccc56)cc4)n[nH]3)c2n1': 12.0,
  'CC(=O)C(c1cccc(C(=O)c2cccc3ccccc23)c1)c1noc(-c2c[nH]nc2C2CCCCC2)n1': 12.0,
  'Fc1cc(Br)c(CNc2nc3cc(-c4cccc5ccccc45)ccc3n2Cc2ccccc2)c(F)c1F': 12.0,
  'O=C(Cl)Cc1cn(-c2ccc3cc(-c4ccccc4F)[nH]c3c2)c2cccnc12': 11.9,
  'CC1(C)Cc2cc3cc(-c4cccc5ccccc45)ccc3nc2CC1=O': 11.8,
  'O=C(CNc1ccc2c(-c3ccc(F)cc3)c(-c3ccccc3)[nH]c2c1)c1cccc2ccccc12': 11.8,
  'Cc1ccc2c(=O)-n(Cc3ccc(NC(=O)C(F)(F)c4cccc5ccccc45)cc3)cnc2c1F': 11.8,
  'Cc1cc(-c2cccc(NC(=O)NC3CCCCCC3(F)F)c2)c2ccccc2c1': 11.8,
  'Nc1nc(-c2ccccc2)c(-c2ccc3cc(C(=O)N4CC5CCC4CN5)[nH]c3c2F)s1': 11.7,
  'Cn1ncc2c1CCCn1c-2nc2ccc(-c3cccc(C(=O)C(C)(F)F)c3)cc2c1=O': 11.7,
  'NCC(=O)c1cccc(C(=O)c2nc(-c3noc(

### Step 3: Evaluate the result

In [15]:
results = {}
results[name] = pred_smiles 

# submission-ready results
t1 = time.time()
out = group.evaluate(results)
print(f"Consumed time: {time.time() - t1:.3f} s")

The input is a dictionary, expected to have SMILES string as key and docking score as value!
---- Calculating average docking scores ----
---- Calculating synthetic accessibility score ----
Found local copy...
---- Calculating molecular filters scores ----
MolFilter is using the following filters:
Rule_Glaxo: True
Rule_PAINS: True
Rule_SureChEMBL: True
---- Calculating diversity ----
---- Calculating novelty ----


Consumed time: 45.081 s


In [16]:
i = 0
out['3pbl']['5000'].keys()

dict_keys(['docking_scores_dict', 'top100', 'top10', 'top1', 'sa_dict', 'sa', 'pass_list', '%pass', 'top1_%pass', 'diversity', 'novelty', 'top smiles'])

In [28]:
print(list(out['3pbl']['5000'].keys())[i])
print(out['3pbl']['5000'][list(out['3pbl']['5000'].keys())[i]])
i += 1

top smiles
['O=c1c2c(F)c(-c3cccc4ccccc34)ccc2ncn1CC12CCC(CC1)OC2', 'O=c1c2c(Br)cccc2ncn1Cc1cc(F)c(-c2n~c3c(C4=NNN=N4)cccc3o2)cc1F', 'O=C1OC(=O)C23CCOCC12N=NN3CC12CC(C3=NC(c4cncc5ccccc45)=NN3)(CO1)C2', 'FC(F)(F)c1cncc(CN2CCc3cc(-c4cccc5ccccc45)ccc32)c1', 'Cc1cc(N)n2ncc(-c3nc(-c4ccc(-c5cccc6ccccc56)cc4)n[nH]3)c2n1', 'CC(=O)C(c1cccc(C(=O)c2cccc3ccccc23)c1)c1noc(-c2c[nH]nc2C2CCCCC2)n1', 'Fc1cc(Br)c(CNc2nc3cc(-c4cccc5ccccc45)ccc3n2Cc2ccccc2)c(F)c1F', 'O=C(Cl)Cc1cn(-c2ccc3cc(-c4ccccc4F)[nH]c3c2)c2cccnc12', 'CC1(C)Cc2cc3cc(-c4cccc5ccccc45)ccc3nc2CC1=O', 'O=C(CNc1ccc2c(-c3ccc(F)cc3)c(-c3ccccc3)[nH]c2c1)c1cccc2ccccc12', 'Cc1ccc2c(=O)-n(Cc3ccc(NC(=O)C(F)(F)c4cccc5ccccc45)cc3)cnc2c1F', 'Cc1cc(-c2cccc(NC(=O)NC3CCCCCC3(F)F)c2)c2ccccc2c1', 'Nc1nc(-c2ccccc2)c(-c2ccc3cc(C(=O)N4CC5CCC4CN5)[nH]c3c2F)s1', 'Cn1ncc2c1CCCn1c-2nc2ccc(-c3cccc(C(=O)C(C)(F)F)c3)cc2c1=O', 'NCC(=O)c1cccc(C(=O)c2nc(-c3noc(-c4ccccc4C4CCCC4)n3)ccc2F)c1', 'CC(F)(F)c1ccccc1-c1ccc2c(c1)C(NC(=O)Nc1cccc3ccccc13)CC2', 'CC(C)n1c(-c2c(F)ccc

### Submit to leaderboard

https://tdcommons.ai/benchmark/overview/#step-by-step-instructions